### Training Model
Code used to create hand writing predictor model.

In [1]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from random import randint

from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from tensorflow.keras.optimizers import SGD

In [2]:
def load_dataset():
    # load dataset
    (train_x, train_y), (testX, testY) = mnist.load_data()

    train_x = train_x.reshape((train_x.shape[0], 28, 28, 1))
    testX = testX.reshape((testX.shape[0], 28, 28, 1))
    # one hot encode targets
    train_y = to_categorical(train_y)
    testY = to_categorical(testY)
    trainX = train_x[:1000]
    trainY = train_y[:1000]
    testX = testX[:300]
    testY = testY[:300]
    return trainX, trainY, testX, testY

# scale pixels
def prep_pixels(train, test):
    train_norm = train.astype('float32')
    test_norm = test.astype('float32')
    # normalize to range 0-1
    train_norm = train_norm / 255.0
    test_norm = test_norm / 255.0

    return train_norm, test_norm

def define_model():
    global model
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 84, 1)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(1000, activation='softmax'))
    # compile model
    opt = SGD(lr=0.01, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Run model fitting
def run_model():
    trainX, trainY, testX, testY = new_trainX, new_trainY, new_testX, new_testY
    # prepare pixel data
    trainX, testX = prep_pixels(trainX, testX)
    # define model
    model = define_model()
    # fit model
    model.fit(trainX, trainY, epochs=5, batch_size=32, verbose=1)

In [3]:
# Load data
trainX, trainY, testX, testY = load_dataset()

In [4]:
# Create new trainX, trainY, testX, testY datasets
new_trainY = np.zeros((15000, 1000))
new_trainX = np.zeros((15000, 28, 84, 1))
new_testY = np.zeros((4500, 1000))
new_testX = np.zeros((4500, 28, 84, 1))

In [5]:
# Fill new train and test datatsets with numbers ranging from 0 to 999 by horizontally concatenating mnist digits together.
for idx in range(5000):
    i = randint(0, 999)
    x_entry = trainX[i].reshape((28, 28, 1))
    x_entry = cv.resize(x_entry, (84, 28)).reshape((28, 84, 1))
    y_index = np.where(trainY[i] == 1)[0][0]
    new_trainX[idx] = x_entry
    new_trainY[idx][y_index] = 1

for idx in range(5000, 10000):
    i = randint(0, 999)
    j = randint(0, 999)
    x_entry = cv.hconcat([trainX[i], trainX[j]]).reshape((28, 56, 1))
    x_entry = cv.resize(x_entry, (84, 28)).reshape((28, 84, 1))
    y_index = np.where(trainY[i] == 1)[0][0]*10 + np.where(trainY[j] == 1)[0][0]
    new_trainX[idx] = x_entry
    new_trainY[idx][y_index] = 1
    
for idx in range(10000, 15000):
    i = randint(0, 999)
    j = randint(0, 999)
    k = randint(0, 999)
    x_entry = cv.hconcat([trainX[i], trainX[j], trainX[k]]).reshape((28, 84, 1))
    x_entry = cv.resize(x_entry, (84, 28)).reshape((28, 84, 1))
    y_index = np.where(trainY[i] == 1)[0][0]*100 + np.where(trainY[j] == 1)[0][0]*10 + np.where(trainY[k] == 1)[0][0]
    new_trainX[idx] = x_entry
    new_trainY[idx][y_index] = 1
    
for idx in range(1500):
    i = randint(0, 299)
    x_entry = testX[i].reshape((28, 28, 1))
    x_entry = cv.resize(x_entry, (84, 28)).reshape((28, 84, 1))
    y_index = np.where(testY[i] == 1)[0][0]
    new_testX[idx] = x_entry
    new_testY[idx][y_index] = 1

for idx in range(1500, 3000):
    i = randint(0, 299)
    j = randint(0, 299)
    x_entry = cv.hconcat([testX[i], testX[j]]).reshape((28, 56, 1))
    x_entry = cv.resize(x_entry, (84, 28)).reshape((28, 84, 1))
    y_index = np.where(testY[i] == 1)[0][0]*10 + np.where(testY[j] == 1)[0][0]
    new_testX[idx] = x_entry
    new_testY[idx][y_index] = 1
    
for idx in range(3000, 4500):
    i = randint(0, 299)
    j = randint(0, 299)
    k = randint(0, 299)
    x_entry = cv.hconcat([testX[i], testX[j], testX[k]]).reshape((28, 84, 1))
    x_entry = cv.resize(x_entry, (84, 28)).reshape((28, 84, 1))
    y_index = np.where(testY[i] == 1)[0][0]*100 + np.where(testY[j] == 1)[0][0]*10 + np.where(testY[k] == 1)[0][0]
    new_testX[idx] = x_entry
    new_testY[idx][y_index] = 1

In [24]:
run_model()

Epoch 1/5
469/469 [==============================] - 42s 89ms/step - loss: 3.3481 - accuracy: 0.4151
Epoch 2/5
469/469 [==============================] - 60s 127ms/step - loss: 2.1155 - accuracy: 0.6274
Epoch 3/5
469/469 [==============================] - 41s 88ms/step - loss: 1.3164 - accuracy: 0.7179
Epoch 4/5
469/469 [==============================] - 44s 94ms/step - loss: 0.7199 - accuracy: 0.8156
Epoch 5/5
469/469 [==============================] - 38s 80ms/step - loss: 0.4184 - accuracy: 0.8821


In [25]:
results = model.evaluate(new_testX, new_testY, batch_size=128)

36/36 [==============================] - 3s 74ms/step - loss: 207.3194 - accuracy: 0.7440


In [26]:
ret = model.predict(new_testX)

### Test predictions of handwritten numbers using train images

In [ ]:
from tensorflow import keras

In [30]:
# Function to find ROI of handwritten number in an image
def handwritingExtract(file_path):
    img = plt.imread(file_path)
    hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)
    h, s, v = cv.split(hsv)
    inverted = s

    ret, th = cv.threshold(inverted, 150, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)
    dilateKernel = np.ones((1, 30), np.uint8)
    dilated = cv.dilate(th, dilateKernel)
    contours, hierarchy = cv.findContours(dilated, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    imgShape = img.shape
    validContours = []
    for contour in contours:
        area = cv.contourArea(contour)
        x, y, w, h = cv.boundingRect(contour)
        if area > 100 and (y > 0.80 * imgShape[0] or y + h < 0.20 * imgShape[0]):
            validContours.append(contour)

    contourImg = np.copy(img)
    cv.drawContours(contourImg, validContours, -1, (0, 255, 0), 3)
    cv.line(contourImg, (0, int(0.8 * imgShape[0])), (imgShape[1], int(0.8 * imgShape[0])), (255, 0, 0))
    cv.line(contourImg, (0, int(0.2 * imgShape[0])), (imgShape[1], int(0.2 * imgShape[0])), (255, 0, 0))

    candidates = []
    for contour in validContours:
        x, y, w, h = cv.boundingRect(contour)
        candidate = img[y:y+h, x:x+w]
        candidates.append(candidate)

    return candidates

In [38]:
# Load an image
candidates = handwritingExtract(r"C:\Users\Asus\Downloads\nus-sds-dsc2021\train_data\train_images\1051.jpg")

In [64]:
# Assumes correct contour containing number is the last contour, then resize it
digit_img = cv.resize(cv.cvtColor(candidates[-1], cv.COLOR_BGR2GRAY),(84, 28)).reshape((28, 84, 1))

In [89]:
# Get predicted value from model
np.where(model.predict(np.array([digit_img]))[0] == 1)[0][0]

40

In [76]:
# Save model for future use
model.save('final_model.h5')

In [83]:
# Loads a saved model
model = keras.models.load_model(r"C:\Users\Asus\Documents\nus-data-science-comp-cv\notebook\final_model.h5")